In [1]:
# Make variables for all files:
folder_datafiles = '/mbshome/nvelthuijs/Cofactors/20200402_Transcriptionregulationlist/Datafiles/'
folder_corrected_names = '/mbshome/nvelthuijs/Cofactors/20200402_Transcriptionregulationlist/Datafiles_corrected_names/'
folder_output = '/mbshome/nvelthuijs/Cofactors/20200402_Transcriptionregulationlist/Output_files/'

# Create file variables for HUGO database:
hugo_file = folder_datafiles + 'hugo_ensg_uniprot_approved.txt'
hugo_previous_file = folder_datafiles + 'hugo_previous_symbols_aliases_approved.txt'

# Datasources, filenames and paths that need name checks:
datasources = ['goa_human', 'humap', 'corum', 'biogrid', 'intact', 'gocofs',
               'tfs', 'crems', 'snfs', 'nursa', 'nvs_complexes', 'nvs_proteins']
filenames = [datasource + '.txt' for datasource in datasources]
filepaths = [folder_datafiles + filename for filename in filenames]

for number, (source, name, path) in enumerate(zip(datasources, filenames, filepaths)):
    print(number, source, '\t', name, path)

0 goa_human 	 goa_human.txt /mbshome/nvelthuijs/Cofactors/20200402_Transcriptionregulationlist/Datafiles/goa_human.txt
1 humap 	 humap.txt /mbshome/nvelthuijs/Cofactors/20200402_Transcriptionregulationlist/Datafiles/humap.txt
2 corum 	 corum.txt /mbshome/nvelthuijs/Cofactors/20200402_Transcriptionregulationlist/Datafiles/corum.txt
3 biogrid 	 biogrid.txt /mbshome/nvelthuijs/Cofactors/20200402_Transcriptionregulationlist/Datafiles/biogrid.txt
4 intact 	 intact.txt /mbshome/nvelthuijs/Cofactors/20200402_Transcriptionregulationlist/Datafiles/intact.txt
5 gocofs 	 gocofs.txt /mbshome/nvelthuijs/Cofactors/20200402_Transcriptionregulationlist/Datafiles/gocofs.txt
6 tfs 	 tfs.txt /mbshome/nvelthuijs/Cofactors/20200402_Transcriptionregulationlist/Datafiles/tfs.txt
7 crems 	 crems.txt /mbshome/nvelthuijs/Cofactors/20200402_Transcriptionregulationlist/Datafiles/crems.txt
8 snfs 	 snfs.txt /mbshome/nvelthuijs/Cofactors/20200402_Transcriptionregulationlist/Datafiles/snfs.txt
9 nursa 	 nursa.txt /m

In [2]:
# Create set of HUGO approved gene names (hugo_genes)
with open(hugo_file) as hugo:
    next(hugo)
    hugo_genes = {line.split('\t')[0] for line in hugo}
    
# Create dictionairy with HUGO names as values and any possible
# wrong names as keys.
hugo_previous_dict = {}

with open(hugo_previous_file) as hugo:
    next(hugo)
    for line in hugo:
        line = line.strip().split('\t')
        if len(line) > 1:
            for names in line[1:]:
                names = names.split(', ')
                for name in names:
                    hugo_previous_dict[name] = line[0]
    hugo_previous_dict.pop('')
    hugo_previous_dict['RYDEN'] = 'SHFL' # Manually added, HUGO has RyDEN as alias, CORUM uses RYDEN.

In [3]:
# Find and change unnaproved names:


def wrong_name_print(prot):
    """
    Checks of protein has a correct name.
    If not, prints the file it is in and
    the incorrect name.
    """
    if not protein in hugo_genes:
        print(source, '\t', protein)

# Open alle datasource files 1 by 1.
with open(folder_output + '01_Uncorrected_erronous_names.txt', 'w') as wrong_names_file:
    remaining_wrong_names = set()
    for source, name, path in zip(datasources, filenames, filepaths):

        # Exact method depends on the datasource.
        # Find and replace incorrect names for humap, corum and biogrid.
        # Wrong names in other files (except IntACT) were manually changed.
        # Both print and write to outputfile all wrong names still in the datasets.
        
        if source == 'goa_human':
            with open(path) as oldfile, open(folder_corrected_names + 'goa_human_corrected_names.txt', 'w') as newfile:
                for oldline in oldfile:
                    if oldline.startswith('!'):
                        newfile.write(oldline)
                        continue
                    splitline = oldline.strip().split('\t')
                    protein = splitline[2]
                    if not protein in hugo_genes:
                        new_name = hugo_previous_dict.get(protein, protein)
                        if not new_name in hugo_genes:
                            if not (source, new_name) in remaining_wrong_names:
                                remaining_wrong_names.add((source, new_name))
                                print(source, new_name)
                                wrong_names_file.write(source + '\t' + new_name + '\n')
                        splitline[2] = new_name
                        newline = '\t'.join(splitline)
                        newfile.write(newline + '\n')
                    else:
                        newfile.write(oldline)
                    
        elif source == 'humap':
            with open(path) as oldfile, open(folder_corrected_names + 'humap_corrected_names.txt', 'w') as newfile:
                for line in oldfile:
                    proteins = line.strip().split('\t')
                    new_proteins = proteins
                    for index, protein in enumerate(proteins):
                        if not protein in hugo_genes:
                            new_name = hugo_previous_dict.get(protein, protein)
                            new_proteins[index] = new_name
                            if not new_name in hugo_genes:
                                if not (source, new_name) in remaining_wrong_names:
                                    remaining_wrong_names.add((source, new_name))
                                    print(source, new_name)
                                    wrong_names_file.write(source + '\t' + new_name + '\n')
                    newline = '\t'.join(new_proteins) + '\n'
                    newfile.write(newline)

        elif source == 'corum':
            with open(path) as oldfile, open(folder_corrected_names + 'corum_corrected_names.txt', 'w') as newfile:
                for oldline in oldfile:
                    line = oldline.strip().split('\t')
                    
                    wrong_name_found = False
                    if line[2] == 'Human':
                        proteins = line[12].strip().split(';')
                        for protein in proteins:
                            if not protein in hugo_genes:
                                wrong_name_found = True
                        
                    if wrong_name_found == False:
                        newfile.write(oldline)
                    else:
                        proteins = line[12].strip().split(';')
                        new_proteins = proteins
                        for index, protein in enumerate(proteins):
                            protein = protein.strip()
                            if not protein in hugo_genes:
                                new_name = hugo_previous_dict.get(protein, protein)
                                new_proteins[index] = new_name
                                if not new_name in hugo_genes and new_name != '':
                                    if not (source, new_name) in remaining_wrong_names:
                                        remaining_wrong_names.add((source, new_name))
                                        print(source, new_name)
                                        wrong_names_file.write(source + '\t' + new_name + '\n')
                        line[12] = ';'.join(new_proteins)
                        newline = '\t'.join(line)
                        newfile.write(newline + '\n')

        elif source == 'biogrid':
            with open(path) as oldfile, open(folder_corrected_names + 'biogrid_corrected_names.txt', 'w') as newfile:
                newfile.write(oldfile.readline())
                
                for line in oldfile:
                    wrong_name_found = False
                    splitline = line.strip().split('\t')
                    if splitline[15] == '9606' and splitline[16] == '9606':
                        prot_a, prot_b = (splitline[7], splitline[8])
                        for index, protein in enumerate([prot_a, prot_b]):
                            if not protein in hugo_genes:
                                splitline[index + 7] = hugo_previous_dict.get(protein, protein)
                                if not splitline[index + 7] in hugo_genes:
                                    if not (source, splitline[index + 7]) in remaining_wrong_names:
                                        remaining_wrong_names.add((source, splitline[index + 7]))
                                        print(source, splitline[index + 7])
                                        wrong_names_file.write(source + '\t' + splitline[index + 7] + '\n')
                    newline = '\t'.join(splitline)
                    newfile.write(newline + '\n')

        elif source == 'intact':
            # For Intact UniProt IDs will be used,
            # so no need to check for wrong names.
            continue

        elif source == 'nvs_proteins':
            with open(path) as file:
                for line in file:
                    protein = line.split('\t')[0]
                    if not protein in hugo_genes:
                        print(source, protein)
                        wrong_names_file.write(source + '\t' + protein + '\n')

        elif source == 'nvs_complexes':
            with open(path) as file:
                for line in file:
                    proteins = line.strip().split('\t')[2:]
                    for protein in proteins:
                        if not protein in hugo_genes:
                            print(source, protein)
                            wrong_names_file.write(source + '\t' + protein + '\n')

        else:
            with open(path) as protein_names:
                # Hand-changed TCP10 to TCP10L3 in gocofs.
                for protein in protein_names:
                    protein = protein.strip()
                    if not protein in hugo_genes:
                        print(source, protein)
                        wrong_names_file.write(source + '\t' + protein + '\n')

goa_human LOC100509620
goa_human A0A087WUV0
goa_human ELOA3CP
goa_human LOC102723996
goa_human A0A096LNW5
goa_human A0A096LPH7
goa_human A0A096LPK6
goa_human A0A0G2JLM5
goa_human A0A0G2JLQ3
goa_human A0A0G2JMP0
goa_human A0A0G2JMS6
goa_human A0A0G2JNG1
goa_human A0A0G2JNH3
goa_human A0A0G2JP03
goa_human A0A0G2JPB7
goa_human A0A0G2JQ47
goa_human A0A0U1RRA0
goa_human A0A0X1KG70
goa_human A0A140TA62
goa_human LOC100996750
goa_human A0A1B0GTG8
goa_human A0A1B0GUL7
goa_human LOC102723971
goa_human A0A1B0GUZ9
goa_human A0A1W2PQM1
goa_human hCG_1642624
goa_human A0A1W2PRN1
goa_human A0A286YEU6
goa_human A0A286YF18
goa_human A0A2R8Y4M4
goa_human A0A2R8Y5M8
goa_human A0A2R8Y747
goa_human A0A2R8Y7Y5
goa_human A0A2R8YE69
goa_human LOC112268384
goa_human A0A2R8YFR7
goa_human A0A3B3IT34
goa_human A0A494C0I6
goa_human LOC728392
goa_human A0A499FJF3
goa_human A0A5F9ZH02
goa_human LOC112267881
goa_human PEDS1
goa_human A6NDX4
goa_human NPEPPSL1
goa_human A6NIZ1
goa_human A6NJY4
goa_human A6NLC8
goa_hu

biogrid BRCA3
biogrid IGKV@
biogrid LOC100505498
biogrid LOC101930123
biogrid PP7080
biogrid LOC100129492
biogrid LOC401010
biogrid LOC727896
biogrid NPCA1
tfs LOC730110
nvs_complexes H2B
nvs_proteins H2B
